# Modelling coupled degradation mechanisms in PyBaMM

This notebook shows how to set up a PyBaMM model in which many degradation mechanisms run at the same time and interact with one another.

In [ ]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm
import matplotlib.pyplot as plt
import pandas as pd
import bokeh.io
from functools import partial
import pybamm
import numpy as np
%load_ext autoreload
%autoreload 2
# this is here only for completeness to clarify where
# the methods are nested (you probably already imported this earlier)
bokeh.io.reset_output()
bokeh.io.output_notebook()


In [ ]:
model = pybamm.lithium_ion.DFN(
    {
        "SEI": "solvent-diffusion limited",
        "SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",  # alias for "SEI porosity change"
        "particle mechanics": ("swelling and cracking", "swelling only"),
        "SEI on cracks": "true",
        "loss of active material": "stress-driven",
        "calculate discharge energy": "true",  # for compatibility with older PyBaMM versions
    }
)

In [ ]:
param = pybamm.ParameterValues("OKane2022")
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator
    "x_p": 5,  # positive electrode
    "r_n": 30,  # negative particle
    "r_p": 30,  # positive particle
}

model.variables["SoC"] = model.variables[
    "Discharge capacity [A.h]"
] / param['Nominal cell capacity [A.h]']

#param.update({"Lower voltage cut-off [V]": 3.6})
param.update({"SEI kinetic rate constant [m.s-1]": 1e-2})
nominal_cell_capacity = param['Nominal cell capacity [A.h]']

Depending on the parameter set being used, the particle cracking model can require a large number of mesh points inside the particles to be numerically stable.

Define a cycling protocol and solve. The protocol from O'Kane et al. [10] is used here, except with 10 ageing cycles instead of 1000.

In [ ]:
N = 30 # number or repetitions
df = pd.read_csv("ussd.txt", sep="\t")
df.power = (df.power / df.power.mean()) * 0.1

df = pd.concat([df]*N, ignore_index=True)

df.power = df.power 
t = df.index
drive_cycle_power = np.column_stack([t, df.power])

In [ ]:
N = 30 # number or repetitions
import pandas as pd
df = pd.read_csv("udds_current.csv")

In [ ]:
df = pd.concat([df]*N, ignore_index=True)

drive_cycle_power = np.column_stack([df.index, -df.C ])
t = df.index
cycle_number = 1

In [ ]:

def SoC_termination(threshold):
    SoC_init = 1
    def SoC(variables):
        return (SoC_init - variables["SoC"]) - threshold
        
    return pybamm.step.CustomTermination(
        name=f"State of Charge Threshold {threshold}", 
        event_function=SoC
    )

In [ ]:
std_exp = pybamm.Experiment(
    [
        "Hold at 4.2 V until C/100 (5 minute period)",
        "Rest for 4 hours (5 minute period)",
        "Discharge at 0.1C until 2.5 V (5 minute period)",  # initial capacity check
        "Charge at 0.3C until 4.2 V (5 minute period)",
        "Hold at 4.2 V until C/100 (5 minute period)",
    ]
    + [
        (
            "Charge at 1C until 4 V",
            "Hold at 4 V until 50 mA",
            "Charge at C/4 until 4.2 V",
            "Hold at 4.2 V until 50 mA",
            pybamm.step.c_rate(0.25, termination = SoC_termination(0.8)),
            pybamm.step.current(drive_cycle_power, termination = SoC_termination(0.2)),
        )
    ]
    * cycle_number
    #+ ["Discharge at 0.1C until 2.5 V (5 minute period)"],  # final capacity check
)

In [ ]:
# load solvers
pybamm.set_logging_level("NOTICE")
safe_solver = pybamm.CasadiSolver( mode="safe")

# create simulations
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=std_exp, var_pts=var_pts, solver=safe_solver)

# solve
sol = safe_sim.solve(showprogress= True, initial_soc= 1)


In [ ]:
from core import Wrap_PyBamm

df = Wrap_PyBamm.get_important_var(sol)

In [ ]:

import plotly.express as px

px.line(df.reset_index(), x=f"Time", y=f"V", color=f"Cycle",height=900)

In [ ]:

px.line(df.reset_index(), x=f"Time", y=f"C", color=f"Cycle",height=900)

In [ ]:
from core import run_iterative_decay
run_iterative_decay(30, 

### TESTING RTOL

In [ ]:
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts, solver=safe_solver)
for rtol in [3, 4, 5, 6, 7]:
    safe_sol = safe_sim.solve(
        solver=pybamm.CasadiSolver(rtol =np.power(10.0,-rtol), mode="safe"),initial_soc=1
    )
    print(
        f"With rtol={rtol}, took {safe_sol.solve_time} "
        + f"(integration time: {safe_sol.integration_time})"
    )


### TESTING ATOL

In [ ]:
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts, solver=safe_solver)
for atol in [3, 4, 5, 6, 7]:
    safe_sol = safe_sim.solve(
        solver=pybamm.CasadiSolver(atol =np.power(10.0,-atol), mode="safe"),initial_soc=1
    )
    print(
        f"With atol={rtol}, took {safe_sol.solve_time} "
        + f"(integration time: {safe_sol.integration_time})"
    )


### TESTING ATOL AND RTOL

In [ ]:
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts, solver=safe_solver)
for tol in [3, 4, 5, 6, 7]:
    safe_sol = safe_sim.solve(
        solver=pybamm.CasadiSolver(atol =np.power(10.0,-tol), rtol =np.power(10.0,-tol), mode="safe"),initial_soc=1
    )
    print(
        f"With atol, rtol={tol}, took {safe_sol.solve_time} "
        + f"(integration time: {safe_sol.integration_time})"
    )

### Testing DT MAX TIME

In [ ]:
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts, solver=safe_solver)
for dt_max in [600, 6000, 60000, 600000]:
    safe_sol = safe_sim.solve(
        solver=pybamm.CasadiSolver(mode="safe", dt_max=dt_max),initial_soc=1
    )
    print(
        f"With dt_max={dt_max}, took {safe_sol.solve_time} "
        + f"(integration time: {safe_sol.integration_time})"
)


### With different Periods

In [ ]:
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts, solver=safe_solver)
safe_sol = safe_sim.solve(
    solver=pybamm.CasadiSolver(mode="safe"),initial_soc=1
)
print(
    f"With exp period default, took {safe_sol.solve_time} "
    + f"(integration time: {safe_sol.integration_time})"
)
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp10, var_pts=var_pts, solver=safe_solver)
safe_sol = safe_sim.solve(
    solver=pybamm.CasadiSolver(mode="safe"),initial_soc=1
)
print(
    f"With exp period 10min, took {safe_sol.solve_time} "
    + f"(integration time: {safe_sol.integration_time})"
)
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp60, var_pts=var_pts, solver=safe_solver)
safe_sol = safe_sim.solve(
    solver=pybamm.CasadiSolver(mode="safe"),initial_soc=1
)
print(
    f"With exp period 60min, took {safe_sol.solve_time} "
    + f"(integration time: {safe_sol.integration_time})"
)
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp120, var_pts=var_pts, solver=safe_solver)
safe_sol = safe_sim.solve(
    solver=pybamm.CasadiSolver(mode="safe"),initial_soc=1
)
print(
    f"With exp period 120min, took {safe_sol.solve_time} "
    + f"(integration time: {safe_sol.integration_time})"
)

### Compare safe vs fast with events

In [ ]:
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts, solver=safe_solver)
safe_sol = safe_sim.solve(
    solver=pybamm.CasadiSolver(mode="safe"),initial_soc=1
)
print(
    f"With safe, took {safe_sol.solve_time} "
    + f"(integration time: {safe_sol.integration_time})"
)
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts, solver=safe_solver)
safe_sol = safe_sim.solve(
    solver=pybamm.CasadiSolver(mode="fast with events"),initial_soc=1
)
print(
    f"With fast with events, took {safe_sol.solve_time} "
    + f"(integration time: {safe_sol.integration_time})"
)


In [ ]:
len(exp_list[6])

### Different Experiments with cycle = 10 

In [ ]:
pybamm.settings.set_smoothing_parameters("exact")
for n, exp in enumerate([exp_list[6]],1):
    safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts)
    safe_sol = safe_sim.solve(
        solver=pybamm.CasadiSolver(mode="safe"),initial_soc=1
    )
    

    print(
        f"With exp {n}, took {safe_sol.solve_time} "
        + f"(integration time: {safe_sol.integration_time})"
    )


# Finishing and Starting EXP

In [ ]:
pybamm.set_logging_level("NOTICE")
N = 30 # number or repetitions
df = pd.read_csv("ussd.txt", sep="\t")
df.power = (df.power / df.power.mean()) * 1

df = pd.concat([df]*N, ignore_index=True)

df.power = df.power 
t = df.index
drive_cycle_power = np.column_stack([t, df.power])

cycle_number = 10
safe_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="safe")

main_exp =  pybamm.Experiment(
    [(
            "Charge at 1C until 4 V",
            "Hold at 4 V until 50 mA",
            "Charge at C/4 until 4.2 V",
            "Hold at 4.2 V until 50 mA",
            pybamm.step.c_rate(0.25, termination = SoC_termination(0.8)),
            pybamm.step.current(drive_cycle_power, termination = SoC_termination(0.2)),
        )] * cycle_number
    
    #+ ["Discharge at 0.1C until 2.5 V (5 minute period)"],  # final capacity check
)
end_exp = pybamm.Experiment([("Discharge at 0.1C until 2.5 V (5 minute period)")])

In [ ]:
from dataclasses import dataclass
@dataclass
class Experiments:
    init: pybamm.Experiment
    main: pybamm.Experiment
    end: pybamm.Experiment

In [ ]:
initial_exp

In [ ]:
initial_exp, main_exp, end_exp = Wrap_PyBamm.define_experiment(10)

In [ ]:
list_df = []
safe_sol = None
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=initial_exp, var_pts=var_pts, solver = safe_solver)
new_solution = safe_sim.solve(
    solver=pybamm.CasadiSolver(mode="safe"), initial_soc=1
)
for i in range(1,4):
    safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=main_exp, var_pts=var_pts, solver = safe_solver)
    safe_sol = safe_sim.solve(starting_solution = new_solution
    )
    print(
        f"With exp {i}, took {safe_sol.solve_time} "
        + f"(integration time: {safe_sol.integration_time})"
    )
    list_df.append(Wrap_PyBamm.get_important_var(safe_sol))
    new_solution = safe_sol.last_state
    safe_sol = None
df = pd.concat(list_df)

In [ ]:
import plotly.express as px

px.line(list_df[1].reset_index(), x=f"Time", y=f"V", color=f"Cycle",height=900)

In [ ]:
list_df

In [ ]:
from core import Wrap_PyBamm
df = Wrap_PyBamm.get_important_var(new_solution)


In [ ]:
new_solution.summary_variables

In [ ]:
pybamm.set_logging_level("NOTICE")
N = 30 # number or repetitions
df = pd.read_csv("ussd.txt", sep="\t")
df.power = (df.power / df.power.mean()) * 1

df = pd.concat([df]*N, ignore_index=True)

df.power = df.power 
t = df.index
drive_cycle_power = np.column_stack([t, df.power])

cycle_number = 50
safe_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="safe")
initial_exp = pybamm.Experiment(
    [
        "Hold at 4.2 V until C/100 (5 minute period)",
        "Rest for 4 hours (5 minute period)",
        "Discharge at 0.1C until 2.5 V (5 minute period)",  # initial capacity check
        "Charge at 0.3C until 4.2 V (5 minute period)",
        "Hold at 4.2 V until C/100 (5 minute period)",
    ]
)
main_exp =  pybamm.Experiment(
    [(
            "Charge at 1C until 4 V",
            "Hold at 4 V until 50 mA",
            "Charge at C/4 until 4.2 V",
            "Hold at 4.2 V until 50 mA",
            pybamm.step.c_rate(0.25, termination = SoC_termination(0.8)),
            pybamm.step.current(drive_cycle_power, termination = SoC_termination(0.2)),
        ) * cycle_number
    ]
    #+ ["Discharge at 0.1C until 2.5 V (5 minute period)"],  # final capacity check
)

safe_sol = None
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=initial_exp, var_pts=var_pts, solver = safe_solver)
safe_sol = safe_sim.solve(
    solver=pybamm.CasadiSolver(mode="safe"), initial_soc=1
)
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=main_exp, var_pts=var_pts, solver = safe_solver)
safe_sol = safe_sim.solve(starting_solution = safe_sol
    )
print(
        f"With exp , took {safe_sol.solve_time} "
        + f"(integration time: {safe_sol.integration_time})"
    )

In [ ]:
new_solution = safe_sol.last_state

short_exp =  pybamm.Experiment(
    [(
            "Charge at 1C until 4 V",
            "Hold at 4 V until 50 mA",
            "Charge at C/4 until 4.2 V",
            "Hold at 4.2 V until 50 mA",
            pybamm.step.c_rate(0.25, termination = SoC_termination(0.8)),
            pybamm.step.current(drive_cycle_power, termination = SoC_termination(0.2)),
        )
    ]
    #+ ["Discharge at 0.1C until 2.5 V (5 minute period)"],  # final capacity check
)
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=short_exp, var_pts=var_pts, solver = safe_solver)
safe_sol = safe_sim.solve(starting_solution = new_solution
    )
print(
        f"With exp , took {safe_sol.solve_time} "
        + f"(integration time: {safe_sol.integration_time})"
    )

In [ ]:
pybamm.set_logging_level("WARNING")
cycle_number = 10
initial_exp = pybamm.Experiment(
    [
        "Hold at 4.2 V until C/100 (5 minute period)",
        "Rest for 4 hours (5 minute period)",
        "Discharge at 0.1C until 2.5 V (5 minute period)",  # initial capacity check
        "Charge at 0.3C until 4.2 V (5 minute period)",
        "Hold at 4.2 V until C/100 (5 minute period)",
    ]
)
cycle_number_exp = 10
N = 30 # number or repetitions
df = pd.read_csv("ussd.txt", sep="\t")
df.power = (df.power / df.power.mean()) * 0.1
df = pd.concat([df]*N, ignore_index=True)
df.power = df.power 
t = df.index
drive_cycle_power = np.column_stack([t, df.power])
exp_list = list();


safe_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="safe")

# create simulations
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts, solver=safe_solver)



for i in range(1, 6):
    safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts, solver=safe_solver)
    safe_sol = safe_sim.solve(initial_soc= 1)
    print(
            f"With {i}, took {safe_sol.solve_time} "
            + f"(integration time: {safe_sol.integration_time})"
        )

In [ ]:
# load solvers
safe_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="safe")


# create simulations
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts, solver=safe_solver)


# solve
safe_sim.solve(showprogress= True, initial_soc= 1)
print(f"Safe mode solve time: {safe_sim.solution.solve_time}")


In [ ]:
pybamm.Experiment(
        [
            "Hold at 4.2 V until C/100 (5 minute period)",
            "Rest for 4 hours (5 minute period)",
            "Discharge at 0.1C until 2.5 V (5 minute period)",  # initial capacity check
            "Charge at 0.3C until 4.2 V (5 minute period)",
            "Hold at 4.2 V until C/100 (5 minute period)",
        ]
        + [
            (
                "Charge at 1C until 4 V",
                "Hold at 4 V until 50 mA",
                "Charge at C/4 until 4.2 V",
                "Hold at 4.2 V until 50 mA",
                pybamm.step.c_rate(0.25, termination = SoC_termination(0.8)),
                pybamm.step.c_rate(drive_cycle_power, termination = SoC_termination(0.2)),
            )
        ]* cycle_number_exp,

In [ ]:
main_exp

In [ ]:
for i in range(1,6):
    print(i)

In [ ]:
from scipy.signal import savgol_filter
yhat = savgol_filter(y, 51, 3)

In [ ]:
# All smoothing parameters can be changed at once
pybamm.settings.set_smoothing_parameters(10)
pybamm.settings.min_max_mode = "smooth"
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts, solver=safe_solver)
safe_sol = safe_sim.solve(
    solver=pybamm.CasadiSolver(mode="fast with events"),initial_soc=1
)
print(
    f"With fast with events, took {safe_sol.solve_time} "
    + f"(integration time: {safe_sol.integration_time})"
)

In [ ]:
safe_sim = pybamm.Simulation(model, parameter_values=param, experiment=exp2, var_pts=var_pts, solver=safe_solver)

In [ ]:
sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts)
sol = sim.solve(showprogress= True, initial_soc= 1)

In [ ]:
df_sol = Wrap_PyBamm.get_important_var(sol)

In [ ]:
df_sol.to_csv("test_degradation5.csv")

Three of the degradation mechanisms - SEI, lithium plating and SEI on cracks - cause loss of lithium inventory (LLI). Plotting the different contributions to LLI against throughput capacity as opposed to cycle number allows them to be considered as continuous variables as opposed to discrete ones.

In [ ]:
# Determine the unique cycle numbers
cycle_numbers = df_sol["Cycle"].unique()


# Filter the DataFrame for Step = 5
df_step_5 = df_sol[df_sol["Cycle"] == 3]

# Get unique cycle values, ensuring they are sorted
cycles = np.sort(cycle_numbers)

# Setup the plot
plt.figure(figsize=(10, 6))

# Colormap for gradually darker lines for higher cycle numbers
cm = plt.get_cmap('Greys')
colors = [cm(0.5 + 0.5 * i / len(cycles)) for i in range(len(cycles))]

# Plot VoC vs SoC for each cycle
for i, cycle in enumerate(cycles):
    df_cycle = df_step_5[df_step_5["Cycle"] == cycle]
    plt.plot(df_cycle['SoC'], df_cycle['VoC'], label=f'Cycle {cycle}', color=colors[i])

plt.xlabel('State of Charge (SoC)')
plt.ylabel('VoC')
plt.title('VoC vs. SoC at Step 5 Across Cycles')
plt.legend(title='Cycle', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import Greys256
import pandas as pd
import numpy as np

# Call output_notebook if you're in a Jupyter notebook to display plots inline
output_notebook()

In [ ]:
# Initialize the Bokeh plot
p = figure(title="VoC vs. SoC at Step 5 Across Cycles", x_axis_label='State of Charge (SoC)', y_axis_label='VoC', sizing_mode="stretch_width", height=400)

# Get unique cycle values, ensuring they are sorted
cycles = np.sort(df_step_5.index.get_level_values('Cycle').unique())

# Generate a subset of the Greys palette that gets darker with higher cycle numbers
# Adjust the slice as needed to fit the number of cycles
num_cycles = len(cycles)
greys = Greys256[128:256:128//num_cycles]

# Plot VoC vs SoC for each cycle
for i, cycle in enumerate(cycles):
    df_cycle = df_step_5.xs(cycle, level='Cycle')
    p.line(df_cycle['SoC'], df_cycle['VoC'], legend_label=f'Cycle {cycle}', line_color=greys[i % len(greys)])

p.legend.title = 'Cycle'

# Show the plot
show(p)

In [ ]:
df_sol.filter(["SoC", "VoC"], axis = 1).set_index("SoC").plot()

In [ ]:
import pandas as pd
pd.set_option('plotting.backend', 'pandas_bokeh')
f = df_sol.filter(["C", "V", "SoC"]).plot_bokeh()


In [ ]:
plt.figure()
plt.plot(Qt, Q_SEI, label="SEI", linestyle="dashed")
plt.xlabel("Throughput capacity [A.h]")
plt.ylabel("Total lithium capacity  [A.h]")
plt.legend()
plt.show()

The capacity loss over 10 cycles is so small that the reversible component of the lithium plating is has a larger effect than all the irreversible mechanisms combined. Most of the irreversible capacity fade that does occur is caused by SEI on cracks.

The stress-driven loss of active material (LAM) mechanism [10,11] is also included, so the three main degradation modes - LLI and LAM in each electrode - can be plotted and compared.

In [ ]:
Qt = sol["Throughput capacity [A.h]"].entries
LLI = sol["Loss of lithium inventory [%]"].entries
LAM_neg = sol["Loss of active material in negative electrode [%]"].entries
LAM_pos = sol["Loss of active material in positive electrode [%]"].entries
plt.figure()
plt.plot(Qt, LLI, label="LLI")
plt.plot(Qt, LAM_neg, label="LAM (negative)")
plt.plot(Qt, LAM_pos, label="LAM (positive)")
plt.xlabel("Throughput capacity [A.h]")
plt.ylabel("Degradation modes [%]")
plt.legend()
plt.show()

Both the reversible and irreversible components of LLI are far greater than LAM for this parameter set.

A key internal variable is the porosity. Pore clogging by SEI, lithium plating and other means can trigger other degradation mechanisms and reduce the rate capability of the cell. If the porosity reaches zero, the cell becomes completely unusable and PyBaMM will terminate the simulation.

In [ ]:
eps_neg_avg = sol["X-averaged negative electrode porosity"].entries
eps_neg_sep = sol["Negative electrode porosity"].entries[-1, :]
eps_neg_CC = sol["Negative electrode porosity"].entries[0, :]
plt.figure()
plt.plot(Qt, eps_neg_avg, label="Average")
plt.plot(Qt, eps_neg_sep, label="Separator", linestyle="dotted")
plt.plot(Qt, eps_neg_CC, label="Current collector", linestyle="dashed")
plt.xlabel("Throughput capacity [A.h]")
plt.ylabel("Negative electrode porosity")
plt.legend()
plt.show()

If you want to see some serious degradation, try re-running the simulation with more ageing cycles, or using param.update({}) to increase the degradation parameters beyond the ranges considered by O'Kane et al. [10]

In [ ]:
selected_n = 1000
Qt = sol["Throughput capacity [A.h]"].entries[-selected_n:]


Q_SEI = sol["Loss of capacity to negative SEI [A.h]"].entries[-selected_n:]
Q_SEI_cr = sol["Loss of capacity to negative SEI on cracks [A.h]"].entries[-selected_n:]
Q_plating = sol["Loss of capacity to negative lithium plating [A.h]"].entries[-selected_n:]
Q_side = sol["Total capacity lost to side reactions [A.h]"].entries[-selected_n:]
Q_LLI = (
    sol["Total lithium lost [mol]"].entries[-selected_n:] * 96485.3 / 3600
)  # convert from mol to A.h
plt.figure()
plt.plot(Qt, Q_SEI, label="SEI", linestyle="dashed")
plt.plot(Qt, Q_SEI_cr, label="SEI on cracks", linestyle="dashdot")
plt.plot(Qt, Q_plating, label="Li plating", linestyle="dotted")
plt.plot(Qt, Q_side, label="All side reactions", linestyle=(0, (6, 1)))
plt.plot(Qt, Q_LLI, label="All LLI")
plt.xlabel("Throughput capacity [A.h]")
plt.ylabel("Capacity loss [A.h]")
plt.legend()
plt.show()